# Laplacian Kernelized Completion (LKC)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp1 = np.fft.fft2(lmbda * z - w) / denominator
    return np.fft.ifft2(temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def flip_mat(X):
    temp = np.append(X, np.flip(X, axis = 1), axis = 1)
    return np.append(temp, np.flip(temp, axis = 0), axis = 0)

def inv_flip_mat(mat):
    dim = mat.shape
    N = int(dim[0] / 2)
    T = int(dim[1] / 2)
    temp = (mat[:, : T] + np.flip(mat[:, T :], axis = 1)) / 2
    return (temp[: N, :] + np.flip(temp[N :, :], axis = 0)) / 2

def LKC(y_true, y, lmbda, gamma, tau_s, tau_t, maxiter = 50):
    dim1, dim2, dim3 = y.shape
    eta = 100 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = np.zeros((2 * dim1, 2 * dim2, dim3))
    flip_y = np.zeros((2 * dim1, 2 * dim2, dim3))
    for t in range(dim3):
        flip_y_true[:, :, t] = flip_mat(y_true[:, :, t])
        flip_y[:, :, t] = flip_mat(y[:, :, t])
    N, T, L = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    # ell_s = laplacian(N, tau_s)
    ell_t = laplacian(T, tau_t)
    ell = np.fft.fft2(np.outer(ell_s, ell_t))
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = np.zeros((N, T, L))
        for t in range(dim3):
            x[:, :, t] = prox_2d(z[:, :, t], w[:, :, t], lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = np.zeros((dim1, dim2, dim3))
        for t in range(dim3):
            y_hat[:, :, t] = inv_flip_mat(x[:, :, t])
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

## HighD Dataset

Hyperparameters:

- $\lambda=10^{-3}NT$
- $\gamma=5\lambda$
- $\tau=3$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_46_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * N * T
gamma = 5 * lmbda
tau_s = 1
tau_t = 3
maxiter = 100
tensor_hat = LKC(dense_tensor, sparse_tensor, lmbda, gamma, tau_s, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'HighD_speed_field_70_LKC_rec_lane{}.png'.format(lane + 1))

10
0.1789196681326922
6.32659004100696

20
0.08717562551523428
4.282228352135211

30
0.07263651166400148
4.04398408661419

40
0.06723205558701513
3.9677812041246456

50
0.06658789351869454
3.9348894689138065

60
0.06604712686611242
3.919585249144595

70
0.0657557630248576
3.902986685904798

80
0.06556757130834065
3.8895851198448534

90
0.06535233880678246
3.8774330371089096

100
0.06521416991895763
3.8664188532781303

Running time: 11 seconds.


## CitySim Dataset

Hyperparameters:

- $\lambda=10^{-4}NT$
- $\gamma=5\lambda$
- $\tau=3$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-4 * N * T
gamma = 5 * lmbda
tau_s = 1
tau_t = 3
maxiter = 100
tensor_hat = LKC(dense_tensor, sparse_tensor, lmbda, gamma, tau_s, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'CitySim_speed_field_70_LKC_rec_lane{}.png'.format(lane + 1))

10
0.17844995152020432
5.538399736054597

20
0.11311095661608908
3.8443881813781435

30
0.10696111956183871
3.718234060503583

40
0.10533269840164043
3.700585274871644

50
0.10534369156257373
3.7028063040416375

60
0.10525044127017526
3.7041320103287863

70
0.10530313554694903
3.706614167237156

80
0.10529471607321207
3.7080666453702946

90
0.10531099851442312
3.7094058983935954

100
0.10531367409037566
3.710403304049755

Running time: 5 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>